# Projeto detecção de ações discrepantes 

A distribuição de ações (processos jurídicos) é realizada diariamente durante todo o ano, de forma eletrônica atualmente. Algumas fraudes podem ocorrer durante este processo, pois alguns advogados possuem a prática de distribuir a mesma ação ou ações com as mesmas partes diversas vezes, com o objetivo que o processo siga para um ator jurídico do seu interesse. Outro fator que pode acontecer são diversas distribuições em um único dia ou dias próximos devido a demissões em massa. Com este projeto, objetivou-se criar um sistema capaz de identificar distribuição de ações em massa. Os dados utilizados neste exemplo são oriundos de uma empresa real e por este motivo nomes que possam identificar a empresa serão substituídos por "xxxxx". 

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import statistics

import carga_s3 as int_s3
import projeto_mari as mari
import process_sqc as int_sqc

In [2]:
settings = {
    'database':'base_xxxxx',
    'user':'postgres',
    'password':'******',
    'host':'localhost',
    'port':'*****'
}

In [7]:
conn = psycopg2.connect(**settings)

In [4]:
df = pd.read_sql("SELECT * FROM processos", con=conn)

In [6]:
df['trimestre'] = df.data_distribuicao_tratada.apply(lambda row: row.quarter)

In [7]:
df['data_distribuicao_tratada'] = df.data_distribuicao_tratada.apply(lambda row: str(row))

In [8]:
df = mari.clear_datas(df)
df['data_tratada'] = mari.get_data(df)
df['ano'] = mari.year_get(df)
df['mes'] = mari.month_get(df)

In [ ]:
#######################TEST##########################

In [14]:
df1 = df[['numero', 'data_tratada', 'ano', 'mes', 'uf', 'comarca_tratada', 'juiz', 'valor_acao', 'vara_tipo', 'vara_numero']]

In [9]:
df1.to_csv("dados_xxxxx_tratados.csv", index=False)

In [8]:
df2 = df1.groupby(['data_tratada']).size().reset_index(name='acoes')
df2.head()

,data_tratada,acoes
0,1998-08-28,1
1,2000-05-19,1
2,2001-01-24,1
3,2002-02-01,1
4,2002-04-25,1


In [10]:
df2.to_csv("dados_xxxxxxx_tratados.csv", index=False)

In [ ]:
####################################################

In [16]:
df1 = df[df['ano'] == '2018']

In [17]:
df_t1 = df1[df1['trimestre'] == 1.0]
df_t2 = df1[df1['trimestre'] == 2.0]
df_t3 = df1[df1['trimestre'] == 3.0]
df_t4 = df1[df1['trimestre'] == 4.0]

In [59]:
def shewhart_chart(df,m):
    factors = pd.read_csv("factors_control_chart.csv")
    
    #Control Limits for the R Chart
    r_mean = df['Ampl'].mean()
    ucl_r = factors['D4'][m-2]*r_mean
    lcl_r = factors['D3'][m-2]*r_mean
    
    #Control Limits for the X Chart
    x_mean = df['Mean'].mean()
    ucl_x = x_mean + factors['A2'][m-2]*r_mean
    lcl_x = x_mean - factors['A2'][m-2]*r_mean
    
    return(ucl_r, ucl_x) 
    
    
def mari_summary(df, name, window=30, step=30):
    """
    Funcao para extrair valores extremos da distribuicao de acoes por estado.
    df = dataframe
    name = coluna que sera filtrada
    window = tamanho da janela movel
    step = tamanho do passo da janela movel - se igual a window temos uma janela tradicional (step <= window)
    """
    
    df6 = pd.DataFrame({})
    comarcas_frames = pd.DataFrame({})
    empty_sample = 0
    
    for i in df[f'{name}'].unique():
        df1 = df[df[f'{name}'] == i]
        df2 = df1.groupby('data_tratada').size().reset_index(name='acoes')
        limit_inf = 0
        limit_sup = window
        n = 0  #numero de janelas
        m = window #numero de amostras contidas na janela
        df4 = pd.DataFrame({})
        
        
        while (limit_sup < len(df2['data_tratada'])):
            df3 = pd.DataFrame({f'{name}':[i],
                                'SumAcoes':df2['acoes'][limit_inf:limit_sup].sum(),
                                'DataRangeMax':df2['data_tratada'][limit_inf]+'><'+df2['data_tratada'][limit_sup],
                                'Mean':[df2['acoes'][limit_inf:limit_sup].mean()],
                                'Ampl':[df2['acoes'][limit_inf:limit_sup].max() - df2['acoes'][limit_inf:limit_sup].min()],
                                'Median':[statistics.median(df2['acoes'][limit_inf:limit_sup])]})
            df4 = pd.concat([df4,df3])
            limit_inf += step
            limit_sup += step
            n += 1
        if not df4.empty:
            if n >= 10:
                ucl_r, ucl_x = shewhart_chart(df4,m)
                df5 = df4[((df4['Mean'] >= ucl_x) | (df4['Ampl'] >= ucl_r)) & (df4['Ampl'] >= 4)]
                df6 = pd.concat([df6, df5])
            else:
                df2['comarca_tratada'] = np.repeat(i, [len(df2.acoes)], axis=0)
                comarcas_frames = pd.concat([comarcas_frames, df2])
                empty_sample += 1
        else:
            df2['comarca_tratada'] = np.repeat(i, [len(df2.acoes)], axis=0)
            comarcas_frames = pd.concat([comarcas_frames, df2])
            empty_sample += 1
        
    comarcas_frames = comarcas_frames[comarcas_frames['acoes']>1] 
    geral_ucl = int_sqc.shewhart_moving_range(comarcas_frames.acoes)
    df_frames = comarcas_frames[comarcas_frames['acoes'] > geral_ucl]
            
    return  df6, df_frames, comarcas_frames

## Teste Trimestre 1-2018

In [60]:
result_t1, resultb_t1, c_t1 = mari_summary(df_t1, 'comarca_tratada', window=3, step=2)

In [61]:
result_t1

,Ampl,DataRangeMax,Mean,Median,SumAcoes,comarca_tratada
0,5,2018-03-02><2018-03-07,5.000000,6,15,NITERÓI
0,4,2018-03-23><2018-03-28,3.000000,3,9,MESQUITA
0,10,2018-03-27><2018-03-31,4.666667,2,14,SÃO GONÇALO


In [62]:
resultb_t1

,data_tratada,acoes,comarca_tratada
0,2018-01-08,3,SANTO ANDRÉ
11,2018-03-09,3,RIO BONITO
6,2018-02-21,3,JAPERI
5,2018-02-07,3,GUAPIMIRIM
2,2018-02-02,4,GOIÂNIA
4,2018-03-28,4,GOIÂNIA
17,2018-03-26,3,Outro
2,2018-02-08,3,SÃO JOSÉ DOS CAMPOS
0,2018-02-05,3,SIMÃO DIAS
0,2018-03-23,4,PRIMAVERA DO LESTE


### Cusum tabular

In [63]:
df_res, H = int_sqc.cusum_standardized(xi=c_t1.acoes)

In [64]:
df_res['comarca_tratada'] = c['comarca_tratada']
df_res[df_res['ci_pos']>H]

,yi,yi-(mean+k),ci_pos,n_pos,(mean+k)-yi,ci_neg,n_neg,comarca_tratada
4,3.583499,3.083499,6.361497,4,-4.083499,0,0,GOIÂNIA
3,-0.398167,-0.898167,5.463331,5,-0.101833,0,0,SALVADOR


### EWMA

In [65]:
df_res, H = int_sqc.ewma_chart(xi=c_t1.acoes)

In [66]:
df_res['comarca_tratada'] = c_t1['comarca_tratada']
df_res[df_res['ucl']>H]

,lcl,ucl,xi,yi,comarca_tratada


In [68]:
mari.get_zoom(df, 'comarca_tratada', 'NITERÓI', '2018-01-01','2018-03-31')

,data_tratada,acoes
0,2018-01-04,1
1,2018-01-05,1
2,2018-01-09,2
3,2018-01-10,1
4,2018-01-11,1
5,2018-01-15,3
6,2018-01-16,1
7,2018-01-18,2
8,2018-01-19,1
9,2018-01-23,3


## Teste Trimestre 2-2018

In [70]:
result_t2, resultb_t2, c_t2 = mari_summary(df_t2, 'comarca_tratada', window=3, step=2)

In [71]:
result_t2

,Ampl,DataRangeMax,Mean,Median,SumAcoes,comarca_tratada
0,6,2018-05-20><2018-05-23,3.666667,3,11,NITERÓI
0,7,2018-06-04><2018-06-07,3.666667,2,11,NITERÓI
0,10,2018-05-11><2018-05-16,5.666667,3,17,SÃO JOÃO DE MERITI
0,4,2018-05-22><2018-05-29,2.666667,2,8,GUARULHOS
0,4,2018-05-28><2018-06-04,2.666667,2,8,GUARULHOS
0,5,2018-04-17><2018-04-20,3.333333,3,10,NILÓPOLIS
0,4,2018-05-03><2018-05-09,3.000000,3,9,ITABORAÍ


In [72]:
resultb_t2

,data_tratada,acoes,comarca_tratada
11,2018-06-25,4,PETRÓPOLIS
11,2018-06-28,5,SÃO JOSÉ DOS CAMPOS
5,2018-06-15,5,BETIM
6,2018-06-18,11,BETIM


#### Cusum tabular

In [73]:
df_res, H = int_sqc.cusum_standardized(xi=c_t1.acoes)
df_res['comarca_tratada'] = c['comarca_tratada']
df_res[df_res['ci_pos']>H]

,yi,yi-(mean+k),ci_pos,n_pos,(mean+k)-yi,ci_neg,n_neg,comarca_tratada
4,3.583499,3.083499,6.361497,4,-4.083499,0,0,GOIÂNIA
3,-0.398167,-0.898167,5.463331,5,-0.101833,0,0,SALVADOR


In [75]:
mari.get_zoom(df, 'comarca_tratada', 'SÃO JOÃO DE MERITI', '2018-04-01','2018-06-31')

,data_tratada,acoes
0,2018-04-02,1
1,2018-04-03,1
2,2018-04-04,3
3,2018-04-05,2
4,2018-04-07,2
5,2018-04-09,1
6,2018-04-10,4
7,2018-04-12,1
8,2018-04-13,4
9,2018-04-14,1


## Teste Trimestre 3 - 2018

In [76]:
result_t3, resultb_t3, c_t3 = mari_summary(df_t3, 'comarca_tratada', window=3, step=2)

In [77]:
result_t3

,Ampl,DataRangeMax,Mean,Median,SumAcoes,comarca_tratada
0,8,2018-08-18><2018-08-23,4.000000,2,12,NILÓPOLIS
0,8,2018-08-22><2018-08-25,4.333333,3,13,NILÓPOLIS
0,7,2018-07-04><2018-07-07,4.000000,3,12,DUQUE DE CAXIAS
0,4,2018-08-20><2018-08-28,2.666667,2,8,ITAGUAÍ


In [78]:
resultb_t3

,data_tratada,acoes,comarca_tratada
0,2018-07-03,3,MIGUEL PEREIRA
6,2018-08-06,3,BARRA MANSA
6,2018-07-30,3,BELO HORIZONTE
5,2018-07-18,3,MOGI DAS CRUZES
11,2018-09-06,3,MOGI DAS CRUZES
10,2018-09-15,3,SALVADOR
11,2018-09-25,3,SALVADOR
9,2018-08-21,4,VINHEDO
5,2018-09-24,3,PAULÍNIA
0,2018-08-29,3,ECOPORANGA


In [21]:
mari.get_zoom(df, 'comarca_tratada', 'NILÓPOLIS', '2018-07-01','2018-09-31')

,data_tratada,acoes
0,2018-07-02,1
1,2018-07-03,3
2,2018-07-04,2
3,2018-07-05,2
4,2018-07-06,1
5,2018-07-08,1
6,2018-07-09,4
7,2018-07-10,2
8,2018-07-12,4
9,2018-07-13,2


## Teste Trimestre 4 - 2018

In [79]:
result_t4, resultb_t4, c_t4 = mari_summary(df_t4, 'comarca_tratada', window=3, step=2)

In [80]:
result_t4

,Ampl,DataRangeMax,Mean,Median,SumAcoes,comarca_tratada
0,4,2018-10-30><2018-11-03,4.666667,4,14,NILÓPOLIS
0,4,2018-12-17><2018-12-20,3.000000,3,9,NILÓPOLIS


In [82]:
resultb_t4

,data_tratada,acoes,comarca_tratada
2,2018-10-10,3,ITAGUAÍ
6,2018-10-19,3,ITAGUAÍ
3,2018-10-17,4,QUEIMADOS
5,2018-10-11,3,Outro
9,2018-12-13,3,SOROCABA
17,2018-12-18,3,SANTOS
13,2018-12-21,3,CABO FRIO
7,2018-11-23,3,RESENDE
6,2018-10-29,3,ARARUAMA
13,2018-12-28,6,GUARUJÁ


#### Cusum tabular

In [84]:
df_res, H = int_sqc.cusum_standardized(xi=c_t4.acoes)
df_res['comarca_tratada'] = c_t4['comarca_tratada']
df_res[df_res['ci_pos']>H]

,yi,yi-(mean+k),ci_pos,n_pos,(mean+k)-yi,ci_neg,n_neg,comarca_tratada
13,5.827897,5.327897,5.327897,1,-6.327897,0,0,GUARUJÁ


#### EWMA

In [86]:
df_res, H = int_sqc.ewma_chart(xi=c_t4.acoes)

df_res['comarca_tratada'] = c_t4['comarca_tratada']
df_res[df_res['ucl']>H]

,lcl,ucl,xi,yi,comarca_tratada


In [16]:
mari.get_zoom(df, 'comarca_tratada', 'NILÓPOLIS', '2018-10-01','2018-12-31')

,data_tratada,acoes
0,2018-10-01,3
1,2018-10-02,1
2,2018-10-03,2
3,2018-10-11,3
4,2018-10-15,4
5,2018-10-16,2
6,2018-10-17,1
7,2018-10-19,2
8,2018-10-22,2
9,2018-10-23,3
